In [1]:
import pandas as pd
import plotly.express as px
import swcol as sc

**Unused**
- `carbon_policies`
- `trans_params`
- `transmission_lines`

**Change Manually**
- `financials`
- `fuels`
- `load_zones`
- `gen_build_costs`
- `gen_build_predetermined`
- `gen_info`
- `Non_fuel_energy_sources`

**Generators from this Notebook**
- [x] `loads`
- [x] `fuel_cost`
- [x] `hydro_capacity_factors`
- [x] `hydro_timeseries`
- [x] `variable_capacity_factors`
- [x] `periods`
- [x] `timepoints`
- [x] `timeseries`

In [2]:
dema_path = '../../data/XM-API/variable_query/2022-12-01_2023-11-30/'
model_path = '../../model/inputs/'
# Grow Rate, how much will the demand grow
grow_rate = 0.03
# Period, how many years in between generated data
period = 1
# Cicles, How many periods
cycles = 2 # 15
# Type, based on which column you want to generate the loads. Default: DemaReal_Sistema
# Must be one of the following: [DemaReal_Sistema],[DemaCome_Sistema],[Gene_Sistema],[GeneIdea_Sistema]
load_type = 'DemaReal_Sistema'
# Base year, timestamps will be generated based on this. Must match with the last date on xm_report.
base_year = 2023
# Timeseries
# Mean percentile
percentile = 0.5
# Percentaje that represents of how much each zone takes from the loads
zone_consume_perc = pd.DataFrame(
    data={
    'Zone': ['Antioquia', 'Caribe', 'Nordeste', 'Oriental', 'Surocciden'],
    'Value': [12.968812, 28.515994, 11.426312, 25.573738, 21.515143]
    })

## Time

In [3]:
timepoints = sc.time.generate(model_path, cycles, period,base_year)

File written ../../model/inputs/timepoints.csv
File written ../../model/inputs/timeseries.csv
File written ../../model/inputs/periods.csv


## Loads

In [4]:
XM_report = sc.loads.melt_xm(dema_path)
XM_report.to_csv(dema_path+'Melted_DemaGen_Sys.csv')

loads = sc.loads.generate_by_periods(
    XM_report, load_type, base_year, grow_rate, period, cycles)

In [5]:
loads = sc.loads.generate_by_zones(loads, zone_consume_perc)

Processing Files:   0%|          | 0/17520 [00:00<?, ?it/s]

In [6]:
# Clusterize
loads = sc.loads.loads_to_timestamps(loads)
sc.loads.plot_multiple_line(
    loads,'Date', 'timestamp', 'demand_mw',
    'Zone', 'Clustered Load Curves by Zone', {'demand_mw':'MW'})

In [7]:
loads = sc.time.save_loads(model_path, loads, timepoints)
loads.head()

File written ../../model/inputs/loads.csv


,LOAD_ZONE,timepoints,zone_demand_mw
1758,Oriental,384,2580.219231
1757,Nordeste,384,1152.837692
1756,Caribe,384,2877.073846
1755,Antioquia,384,1308.466154
1759,Surocciden,384,2170.735385


In [8]:
sc.loads.plot_multiple_line(
    loads,'timepoints', 'timepoints', 'zone_demand_mw',
    'LOAD_ZONE', 'Clustered Load Curves by Zone', {'LOAD_ZONE':'Zones',
                                                   'zone_demand_mw':'Demand (MW)',
                                                   'timepoints':'Timepoints'})

## Variable Capacitors
**Wind and Solar**

In [9]:
variable_capacity = sc.variable_capacitors.generate_base_year(
    '../../model/inputs/', False, base_year, timepoints)

variable_capacity = sc.variable_capacitors.generate(
    '../../model/inputs/', variable_capacity)

Variable Capacity Factors saved in ../../model/inputs/variable_capacity_factors.csv


**Water**

In [10]:
import swcol as sc
generation = sc.hydro_capacitors.get_xm_reports()
print(generation['Value'].sum())
generation.head(3)

185211810.9317301


,Values_Name,Value,Datetime
0,EL POPAL,19.85625,2023-11-26
1,LA REBUSCA,0.65040,2023-11-26
2,BAJO TULUA,9.51840,2023-11-26


In [11]:
generation = sc.hydro_capacitors.to_swcol_names(model_path, generation)
generation.head(3)

,Name,Value,Datetime
0,H_ElPopal,19.85625,2023-11-26
1,H_TuluaBajo,9.51840,2023-11-26
2,H_Alejandria,7.32500,2023-11-26


In [12]:
generation = sc.hydro_capacitors.cluster(generation, base_year)
generation.head(3)

,Name,timestamp,Value
0,H_AguaFresca,2023_Q1_holidays_0h,6.833046
1,H_AguaFresca,2023_Q1_holidays_10h,6.687108
2,H_AguaFresca,2023_Q1_holidays_11h,6.672831


In [13]:
generation = sc.hydro_capacitors.normalize(model_path, generation)
generation.head(3)

,GENERATION_PROJECT,timepoint_id,gen_max_capacity_factor
0,H_AguaFresca,25,0.937318
1,H_AguaFresca,35,0.917299
2,H_AguaFresca,36,0.915340


In [14]:
generation = sc.hydro_capacitors.generate(model_path, generation, period, cycles)
generation.head(3)

Hydro Variable Capacitors concatenated with ../../model/inputs/variable_capacity_factors.csv


,GENERATION_PROJECT,timepoint_id,gen_max_capacity_factor
0,E_GuajiraI,1,0.842105
1,E_GuajiraI,10,0.798857
2,E_GuajiraI,100,0.772581


In [15]:
import plotly.express as px
import pandas as pd
plot_hydro_cap = generation[generation['GENERATION_PROJECT'] == 'H_Amaime'].copy()
plot_hydro_cap = plot_hydro_cap.sort_values(by='timepoint_id')
plot_hydro_cap = plot_hydro_cap[plot_hydro_cap['timepoint_id'] <= 192]

timepoints = pd.read_csv(model_path+'timepoints.csv')

plot_hydro_cap = pd.merge(
    plot_hydro_cap, timepoints,
    on='timepoint_id', how="inner")
fig = px.line(
    plot_hydro_cap, 
    x='timestamp', y='gen_max_capacity_factor', title='Capacidad por H_AguaFresca'
)
fig.show()

## Fuel Costs

In [16]:
fuel_costs = sc.fuel_costs.generate(model_path, base_year, cycles, period)
fuel_costs.head()

Fuel cost data saved to ../../model/inputs/fuel_cost.csv


,load_zone,fuel,period,fuel_cost
0,Caribe,GASIMPOR,2023,12.716956
28,Caribe,GASNACIO,2023,7.231772
56,Nordeste,GASNACIO,2023,4.615200
84,Antioquia,GASNACIO,2023,7.231772
112,Caribe,CARBON,2023,4.133101


## Hydro Timeseries

In [17]:
import swcol as sc
generation_xm = sc.hydro_timeseries.get_xm_reports()
generation_xm

,Values_Name,Value,Datetime
0,EL POPAL,19.85625,2023-11-26 00:00:00
1,LA REBUSCA,0.65040,2023-11-26 00:00:00
2,BAJO TULUA,9.51840,2023-11-26 00:00:00
3,AUTOG ARGOS EL CAIRO,6.60891,2023-11-26 00:00:00
4,LA FRISOLERA,0.28129,2023-11-26 00:00:00
...,...,...,...
3397723,TUNJITA,NaN,2020-12-30 23:00:00
3397724,UNION,0.41976,2020-12-30 23:00:00
3397725,URRA,85.62314,2020-12-30 23:00:00
3397726,LA VUELTA,4.21300,2020-12-30 23:00:00


In [18]:
generation = sc.hydro_timeseries.to_swcol_names(model_path, generation_xm, base_year)
generation

,Values_Name,Value,Datetime,Name,Best Match
0,LA REBUSCA,0.65040,2023-11-26 00:00:00,H_LaRebusca,LA REBUSCA
1,LA FRISOLERA,0.28129,2023-11-26 00:00:00,H_Frisolera,LA FRISOLERA
2,GUAVIO MENOR,9.99076,2023-11-26 00:00:00,H_Guaviomeno,GUAVIO MENOR
3,PORCE III MENOR,1.89816,2023-11-26 00:00:00,H_Porce3Meno,PORCE III MENOR
4,MAGALLO,2.82920,2023-11-26 00:00:00,H_Magallo,MAGALLO
...,...,...,...,...,...
3019051,TUNJITA,NaN,2020-12-30 23:00:00,H_Tunjita,TUNJITA
3019052,UNION,0.41976,2020-12-30 23:00:00,H_Union,UNION
3019053,URRA,85.62314,2020-12-30 23:00:00,URRA,URRA
3019054,LA VUELTA,4.21300,2020-12-30 23:00:00,H_La_Vuelta,LA VUELTA


In [19]:
generation = sc.hydro_timeseries.cluster(generation, base_year, percentile)
generation

,hydro_project,timeseries,hydro_avg_flow_mw,hydro_min_flow_mw
0,AMOYA,2023_Q1_holidays,40.37711,0.53926
1,AMOYA,2023_Q1_labor,38.59281,0.00585
2,AMOYA,2023_Q2_holidays,63.70993,0.02993
3,AMOYA,2023_Q2_labor,57.91185,0.00303
4,AMOYA,2023_Q3_holidays,68.49121,1.40060
...,...,...,...,...
1075,H_IquiraII,2023_Q4_holidays,0.00000,0.00000
1076,H_Oibita,2023_Q1_labor,0.00000,0.00000
1077,H_Oibita,2023_Q1_holidays,0.00000,0.00000
1078,H_SBartolome,2023_Q1_labor,0.00000,0.00000


In [20]:
generation = sc.hydro_timeseries.generate(
    model_path,generation, period, cycles, base_year)
generation

Hydro Timeseries saved in ../../model/inputs/hydro_timeseries.csv


,hydro_project,timeseries,hydro_avg_flow_mw,hydro_min_flow_mw
0,AMOYA,2023_Q1_holidays,40.37711,0.53926
1,AMOYA,2023_Q1_labor,38.59281,0.00585
2,AMOYA,2023_Q2_holidays,63.70993,0.02993
3,AMOYA,2023_Q2_labor,57.91185,0.00303
4,AMOYA,2023_Q3_holidays,68.49121,1.40060
...,...,...,...,...
2155,H_IquiraII,2024_Q4_holidays,0.00000,0.00000
2156,H_Oibita,2024_Q1_labor,0.00000,0.00000
2157,H_Oibita,2024_Q1_holidays,0.00000,0.00000
2158,H_SBartolome,2024_Q1_labor,0.00000,0.00000
